# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute( '''
                CREATE KEYSPACE IF NOT EXISTS udacity 
                with replication = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
               ''')
except Exception as e:
    print (e)

#### Set Keyspace

In [8]:
#  Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print (e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Datamodel has to be created on basis of below query

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [9]:
##  Query 1:  Create table song_details
create_table_1 = "CREATE TABLE IF NOT EXISTS song_details"
create_table_1 = create_table_1 + (""" (session_id INT, item_in_session INT, artist TEXT, song_title TEXT,
                                        song_length FLOAT, PRIMARY KEY (session_id, item_in_session)
                )""")
try:
    session.execute(create_table_1)
except Exception as e:
    print(e)





                    

In [10]:
## insert data in song_details

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = " INSERT INTO song_details(session_id,item_in_session,artist,song_title,song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
       
        try:
            session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))
        except Exception as e:
            print(e)


#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## Verify the data for query 1

try:
    rows = session.execute("select artist, song_title, song_length from song_details where session_id=338 and item_in_session=4")
    for row in rows:
        print(" artist : {}\n song_title : {}\n song_length : {}".format(row.artist,row.song_title,row.song_length))
except Exception as e:
    print(e)




 artist : Faithless
 song_title : Music Matters (Mark Knight Dub)
 song_length : 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and 
user (first and last name) for userid = 10, sessionid = 182

In [12]:
##  Query 2:  Create table artist_details

create_table_2 = "CREATE TABLE IF NOT EXISTS artist_details"
create_table_2 = create_table_2 + (""" ( user_id INT,item_in_session INT,session_id INT,  artist TEXT, song_title TEXT, 
                                    first_name TEXT, last_name TEXT, PRIMARY KEY ((user_id, session_id), item_in_session)
                                    )""")

try:
    session.execute(create_table_2)
except Exception as e:
    print(e)

                    

In [13]:
## insert data in artist_details
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = " INSERT INTO artist_details(user_id,item_in_session,session_id,artist,song_title,first_name,last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        try:
            session.execute(query, (int(line[10]),int(line[3]),int(line[8]),line[0],line[9],line[1],line[4]))
        except Exception as e:
            print(e)

In [14]:
## Verify the data for query 2

try:
    rows = session.execute("select artist,song_title,first_name,last_name from artist_details where user_id = 10 and session_id = 182")
    for row in rows:
        print(" artist : {}\n song_title : {}\n user : {}{}".format(row.artist,row.song_title,row.first_name,row.last_name))
except Exception as e:
    print(e)

 artist : Down To The Bone
 song_title : Keep On Keepin' On
 user : SylvieCruz
 artist : Three Drives
 song_title : Greece 2000
 user : SylvieCruz
 artist : Sebastien Tellier
 song_title : Kilometer
 user : SylvieCruz
 artist : Lonnie Gordon
 song_title : Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
 user : SylvieCruz


Query 3 :Give me every user name (first and last) in my music app history who listened to the song 
'All Hands Against His Own'

In [15]:
##  Query 3:  Create table user_details

create_table_3 = "CREATE TABLE IF NOT EXISTS user_details"
create_table_3 = create_table_3 + (""" ( song_title TEXT, user_id INT, first_name TEXT, last_name TEXT , PRIMARY KEY (song_title,user_id))""")

try:
    session.execute(create_table_3)
except Exception as e:
    print(e)

                    

In [16]:
## insert data in user_details
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = " INSERT INTO user_details(song_title,user_id,first_name,last_name)"
        query = query + "VALUES (%s, %s, %s,%s)"
        try:
            session.execute(query, (line[9],int(line[10]),line[1],line[4]))
        except Exception as e:
            print(e)

In [17]:
## Verify the data for query 3


try:
    rows = session.execute("select first_name,last_name from user_details where song_title = 'All Hands Against His Own' ")
    for row in rows:
        print(" username: {} {}".format(row.first_name,row.last_name))
except Exception as e:
    print(e)

 username: Jacqueline Lynch
 username: Tegan Levine
 username: Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions

In [18]:
try:
    session.execute('drop table song_details')
    session.execute('drop table artist_details')
    session.execute('drop table user_details')
except Exception as e:
    print(e)    

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()